In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyquorra.platemap import PlatemapList, WellPosition, PlatemapSize#, PlatemapClient

import os
os.chdir("C:/Users/bvogler/repos/research-and-analysis")

from lunatic import LunaticParser, unpack_annotations
from util import PALETTE


file_plate_pairings = {
    # "lunatic/data/2024-12-13_143852_401155_LUNA_2024-12-13_MicroBCA.xlsx"
    # "lunatic/data/2024-12-17_164941_401155_LUNA_2024-12-17_MicroBCA_left.xlsx"
    # "lunatic/data/2024-12-17_170553_401155_LUNA_2024-12-17_MicroBCA_right.xlsx"
    # "lunatic/data/2025-01-15_144936_401155_LUNA_uBCA.xlsx"
    # "lunatic/data/2025-01-15_161155_401155_LUNA_uBCA.xlsx"
    # "G0033171": "lunatic/data/2025-01-21_112225_401155_LUNA.xlsx",
    # "G0033294": "lunatic/data/2025-01-24_171715_401155_LUNA_2025-01-24_Micro_BCA.xlsx",
    "G0033355": "lunatic/data/2025-01-28_162443_401155_LUNA_2025-01-28_Micro_BCA.xlsx",
    # "G0033458": "lunatic/data/2025-01-30_124501_401155_LUNA.xlsx",
}

In [4]:
lunatic_parser = LunaticParser()
parsed_data = {
    plate_id: lunatic_parser.parse(file_path)
    for plate_id, file_path in file_plate_pairings.items()
}

lunatic_data = {
    plate_id: unpack_annotations(
        pd.concat(
            [pd.DataFrame(plate.wells) for plate in plate_data.plates]
        )  # Note: there is only ever one plate
    )
    for plate_id, plate_data in parsed_data.items()
}

In [3]:
# platemaps = PlatemapList.from_quorra(list(lunatic_data.keys()))
# platemap_data = {
#     platemap.id: unpack_annotations(pd.DataFrame(platemap.well_records))
#     for platemap in platemaps
# }

In [5]:
platemap_size = PlatemapSize.WELLS_96
platemap_data = pd.DataFrame(platemap_size.get_well_positions())
platemap_data["well"] = [str(WellPosition(row, col)) for row, col in zip(platemap_data["row"], platemap_data["column"])]
platemap_data["control"] = None
platemap_data= {"G0033355": platemap_data}

In [6]:
data = pd.concat(
    [
        pd.merge(
            lunatic_data[plate_id],
            platemap_data[plate_id],
            left_on=["row", "column"],
            right_on=["row", "column"],
            how="inner",
        )
        for plate_id in file_plate_pairings.keys()
    ]
)

# add mean of columns A555-569 and A642-664
data = data.assign(
    mean_560=lambda x: x.loc[:, "555":"569"].mean(axis=1),
    mean_655=lambda x: x.loc[:, "642":"664"].mean(axis=1),
    difference=lambda x: x["mean_560"] - x["mean_655"],
)
data.to_csv("lunatic/data/2025-01-28_162443_401155_LUNA_2025-01-28_Micro_BCA.csv", index=False)

In [7]:
# set "control" to "standard" for wells E1:H6, and the rest to None
data["control"] = np.where(
    (data["row"]>3) & (data["column"].isin(range(1, 7))),
    "standard",
    data["control"],
)

In [8]:

def get_limit_of_quantitation(
    df: pd.DataFrame, signal_column: str = "difference"
) -> float:
    """Calculate the limit of quantitation for a given dataframe."""
    blanks = df[df["control"] == "blank"]
    if blanks.empty:
        blanks = df[(df["control"] == "standard") & (df["concentration"] == 0)]
    return blanks[signal_column].mean() + 10 * blanks[signal_column].std()


calibration_data = data[data["control"].isin(["standard", "blank"])]


In [ ]:
# dilution number goes
calibration_data["dilution_no"] 

NameError: name 'calibration_data' is not defined

In [ ]:

limit_of_quantitation_signal = get_limit_of_quantitation(calibration_data)
